In [ ]:
import cv2
import os
import torch
import shutil
from torchvision import transforms
from PIL import Image
from torch.utils.data import DataLoader

# Load the trained model
def load_model(model_path, device):
    model = models.resnet18(pretrained=False)
    num_ftrs = model.fc.in_features
    model.fc = torch.nn.Linear(num_ftrs, 2)  # Assuming 2 classes: kill and no_kill
    model.load_state_dict(torch.load(model_path, map_location=device))
    model = model.to(device)
    model.eval()  # Set to evaluation mode
    return model

# Preprocess a frame for the model (resize, normalize, etc.)
def preprocess_frame(frame):
    preprocess = transforms.Compose([
        transforms.ToPILImage(),
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
    frame_tensor = preprocess(frame).unsqueeze(0)  # Add batch dimension
    return frame_tensor

# Function to classify a video based on the kill banner detection in frames
def classify_video(model, device, video_path, frame_interval=30):
    cap = cv2.VideoCapture(video_path)
    frame_count = 0
    kill_detected = False

    while True:
        success, frame = cap.read()
        if not success:
            break
        
        frame_count += 1
        if frame_count % frame_interval == 0:  # Only process every nth frame
            # Preprocess the frame
            frame_tensor = preprocess_frame(frame).to(device)
            
            # Run the frame through the model
            with torch.no_grad():
                outputs = model(frame_tensor)
                _, pred = torch.max(outputs, 1)
            
            # If a kill is detected in any frame, classify the video as 'kill'
            if pred.item() == 0:  # Assuming '0' is the kill class
                kill_detected = True
                break  # Stop processing further frames as kill is detected

    cap.release()  # Release the video capture
    return kill_detected

# Function to process all video clips in a folder and classify them
def process_video_clips(input_folder, output_kill_folder, output_no_kill_folder, model, device):
    # Ensure the output folders exist
    if not os.path.exists(output_kill_folder):
        os.makedirs(output_kill_folder)
    if not os.path.exists(output_no_kill_folder):
        os.makedirs(output_no_kill_folder)

    # Use os.walk to traverse through all video files in the folder
    for root, dirs, files in os.walk(input_folder):
        for video_file in files:
            if video_file.endswith(('.mp4', '.avi', '.mkv', '.mov')):  # Add other formats as needed
                video_path = os.path.join(root, video_file)
                print(f"Processing video: {video_file}")

                # Classify the video based on the presence of a kill banner
                kill_detected = classify_video(model, device, video_path)

                # Determine the destination folder based on classification
                if kill_detected:
                    destination_folder = output_kill_folder
                    print(f"Copying {video_file} to 'kill' folder.")
                else:
                    destination_folder = output_no_kill_folder
                    print(f"Copying {video_file} to 'no_kill' folder.")
                
                # Ensure no overwriting by generating a unique filename
                unique_name = get_unique_filename(destination_folder, video_file)
                destination_path = os.path.join(destination_folder, unique_name)

                # Copy the video to the determined destination folder
                shutil.copy2(video_path, destination_path)

# Helper function to generate unique filenames
def get_unique_filename(destination_folder, filename):
    base_name, extension = os.path.splitext(filename)
    counter = 1
    unique_name = filename
    # Append a counter to the filename if the file already exists
    while os.path.exists(os.path.join(destination_folder, unique_name)):
        unique_name = f"{base_name}_{counter}{extension}"
        counter += 1
    return unique_name


# Example usage
input_folder = r'path to\POV\POV user'  # Folder containing video clips
output_kill_folder = r'path to\kill'  # Folder for classified 'kill' videos
output_no_kill_folder = r'path to\no_kill'  # Folder for classified 'no_kill' videos
model_path = 'kill_banner_model.pth'  # Path to the trained model

# Set device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Load the trained model
model = load_model(model_path, device)

# Process all video clips and classify them
process_video_clips(input_folder, output_kill_folder, output_no_kill_folder, model, device)


In [ ]:
# get kill frame
# Function to classify a video based on the kill banner detection in frames and extract the first kill frame
def classify_and_extract_kill_frame(model, device, video_path, frame_interval=60, output_frame_folder='kill_frames'):
    cap = cv2.VideoCapture(video_path)
    frame_count = 0
    kill_frame_extracted = False

    # Ensure the output folder exists
    if not os.path.exists(output_frame_folder):
        os.makedirs(output_frame_folder)

    while True:
        success, frame = cap.read()
        if not success:
            break

        frame_count += 1
        if frame_count % frame_interval == 0:  # Only process every nth frame (e.g., every second for 60 FPS)
            # Preprocess the frame
            frame_tensor = preprocess_frame(frame).to(device)

            # Run the frame through the model
            with torch.no_grad():
                outputs = model(frame_tensor)
                _, pred = torch.max(outputs, 1)

            # If a kill is detected, save the frame and stop further processing
            if pred.item() == 0:  # Assuming '0' is the kill class
                kill_frame_extracted = True
                # Save the frame as an image
                frame_filename = os.path.join(output_frame_folder, f"{os.path.basename(video_path)}_frame_{frame_count}.jpg")
                cv2.imwrite(frame_filename, frame)
                print(f"Kill frame extracted and saved at: {frame_filename}")
                break  # Stop processing once the first kill frame is found

    cap.release()  # Release the video capture
    return kill_frame_extracted

# Function to process all video clips in a folder and extract kill frames
def process_video_clips(input_folder, output_frame_folder, model, device, frame_interval=60):
    # Traverse through all video files in the folder
    for root, dirs, files in os.walk(input_folder):
        for video_file in files:
            if video_file.endswith(('.mp4', '.avi', '.mkv', '.mov')):  # Add other formats as needed
                video_path = os.path.join(root, video_file)
                print(f"Processing video: {video_file}")

                # Classify the video and extract the first kill frame
                classify_and_extract_kill_frame(
                    model, device, video_path, frame_interval=frame_interval, output_frame_folder=output_frame_folder
                )

# Example usage
input_folder = r'path to\POV\POV user'  # Folder containing video clips
output_frame_folder = r'path to\kill_frames'  # Folder to save extracted kill frames
model_path = 'kill_banner_model.pth'  # Path to the trained model

# Set device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Load the trained model
model = load_model(model_path, device)

# Process all video clips and extract kill frames, adjusting for 60 FPS video
process_video_clips(input_folder, output_frame_folder, model, device, frame_interval=60)
